In [ ]:
import pandas as pd
import polars as pl
import requests
from bs4 import BeautifulSoup
from pathlib import Path
import re
from tqdm.notebook import tqdm
from time import sleep
from random import uniform
from zipfile import is_zipfile

base = Path.home() / "Local_Workspace" / "Datasets" / "ARPA" / "TRENTINO" / "trento"

In [2]:
stat1 = pd.read_xml(base / "meteostations.xml")
stat2 = pd.read_xml(base / "meteostations2.xml")

In [3]:
stat_ids = stat2["codice"].str.lower().to_list()

In [6]:
def payload(cod_stazione, data_inizio="01/01/2000", data_fine="01/01/2023"):
    return {
        "co": cod_stazione,
        "v": "400.00_400.00,400.55_400.55,400.56_400.56",
        "vn": "Temperatura aria (gradi Celsius),Temperatura aria (gradi Celsius) Min da Annale Idrologico,Temperatura aria (gradi Celsius) Max da Annale Idrologico",
        "p": "Altro,1,1,custom,1",
        "o": "Download,download",
        "i": "Giornaliera,Day,1",
        "cat": "rs",
        "d1": data_inizio,
        "d2": data_fine,
    }

base_url = "http://storico.meteotrentino.it/cgi/webhyd.pl"

def download_data(cod_stazione, path, data_inizio="01/01/2000", data_fine="01/01/2023"):
    with requests.Session() as s:
        r = s.get(base_url, params=payload(cod_stazione, data_inizio, data_fine))
        archive_url = re.search(r"(http://.+.zip\?\d+)", r.text).group(0)
        with s.get(archive_url, stream=True) as archive:
            with open(path, "wb") as f:
                for chunk in archive.iter_content(chunk_size=1024*1024):
                    if chunk:
                        f.write(chunk)


In [7]:
def fragment_path(cod_stazione):
    path =  base / "fragments" / f"{cod_stazione}.zip"
    if not path.parent.exists():
        path.parent.mkdir(parents=True)
    return path

for sid in tqdm(stat_ids):
    path = fragment_path(sid)
    if path.exists() and is_zipfile(path):
        continue
    try:
        download_data(sid, path, "01/01/2000", "01/01/2023")
    except:
        print(f"Error with {sid}. Continuing...")
    sleep(uniform(1, 5))

  0%|          | 0/207 [00:00<?, ?it/s]

In [55]:
download_data("t0308", "01/01/2000", "01/01/2001")

In [14]:
html = BeautifulSoup(r.text, "html.parser")

In [22]:
scripts = html.find_all("script")

In [50]:
import re
rgx = r"(http://.+.zip\?\d+)"
re.search(rgx, r.text).group(0)

'http://storico.meteotrentino.it/wgen/users//545074462/t0367_20231220.zip?20231220224120'

In [ ]:
"'http://storico.meteotrentino.it/wgen/users//545074462/t0367_20231220.zip?20231220224120\';"


In [ ]:
"http://storico.meteotrentino.it/cgi/webhyd.pl?co=t0308&v=400.00_400.00&vn=Temperatura aria (gradi Celsius) &p=Tutti i dati,01/01/1800,01/01/1800,period,1&o=Download,download&i=Giornaliera,Day,1&cat=rs&1703107276110="

In [ ]:
"http://storico.meteotrentino.it/cgi/webhyd.pl?co=t0367&v=400.55_400.55,400.56_400.56&vn=Temperatura aria (gradi Celsius) Min da Annale Idrologico,"


In [ ]:
"Temperatura aria (gradi Celsius) Max da Annale Idrologico&p=Altro,1,1,custom,1&o=Download,download&i=Giornaliera,Day,1&cat=rs&d1=01/01/2000&d2=01/01/2023&1703107710318="